In [59]:
from imblearn.over_sampling import SMOTE
import pandas as pd 
import numpy as np
from sklearn.preprocessing import LabelEncoder
from random import randrange, uniform
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, recall_score

In [98]:
#Read in data
data = pd.read_csv("03 - Cleaned Data Space/Temp_Data.csv")
data.columns

Index(['Unnamed: 0', 'week', 'season', 'Team', 'opponent', 'code', 'date',
       'win_loss', 'OT', 'OT_num', 'pass', 'rush', 'rec', 'points_allowed',
       'points_scored', 'point_differential', 'home_game', 'CONF', 'SOR',
       'FPI', 'SOS', 'GC', 'AVGWP', 'AP_rank', 'opponent_rank', 'rank_change',
       'ranked_opponent', 'Elo_pregame'],
      dtype='object')

In [99]:
data = data[["AP_rank", "rank_change", "win_loss", "OT", "OT_num", "point_differential", "home_game", "CONF", "opponent_rank", "Elo_pregame"]]

data.head()

,AP_rank,rank_change,win_loss,OT,OT_num,point_differential,home_game,CONF,opponent_rank,Elo_pregame
0,3,-1,1,0,0,6,1,SEC,NaN,1618.269758
1,12,-9,0,0,0,-17,0,SEC,13.0,1560.283091
2,14,-4,0,0,0,-2,0,Big 12,NaN,1594.035405
3,2,0,1,0,0,8,1,American,NaN,1653.478999
4,21,-3,0,0,0,-2,1,Sun Belt,NaN,1641.639350


In [100]:
#create a variable for if a ranked team loses to an unranked team 
data["upset"] = (data["win_loss"] == 0) & (np.isnan(np.array(data["opponent_rank"]))) & (np.isnan(np.array(data["AP_rank"])) == False)

In [101]:
#Making sure the data is all numeric to set up for smote 
le = LabelEncoder()
data["CONF"] = le.fit_transform(data["CONF"])


In [64]:
data.head()

,AP_rank,rank_change,win_loss,OT,OT_num,point_differential,home_game,CONF,opponent_rank,Elo_pregame,upset
0,3,-1,1,0,0,6,1,9,NaN,1618.269758,False
1,12,-9,0,0,0,-17,0,9,13.0,1560.283091,False
2,14,-4,0,0,0,-2,0,2,NaN,1594.035405,True
3,2,0,1,0,0,8,1,1,NaN,1653.478999,False
4,21,-3,0,0,0,-2,1,10,NaN,1641.639350,True


In [102]:
#Training the initial random forest 
df = data.drop(["AP_rank", "upset"], axis = 1)

In [103]:
X = df.drop(['rank_change'], axis=1)
y = df['rank_change']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)


In [104]:
np.mean(np.array(abs(y_pred - y_test)))

np.float64(2.94273127753304)

In [105]:
data["upset"].value_counts()

upset
False    984
True     147
Name: count, dtype: int64

In [ ]:
data["upset"] = data["upset"].replace({True: 1, False: 0})


/var/folders/yr/gdw8wx851dl8qy0bnbd7gt6w0000gq/T/ipykernel_11692/1182948425.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data["upset"] = data["upset"].replace({True: 1, False: 0})


upset
0    984
1    147
Name: count, dtype: int64

In [110]:
data.head()

,AP_rank,rank_change,win_loss,OT,OT_num,point_differential,home_game,CONF,opponent_rank,Elo_pregame,upset
0,3,-1,1,0,0,6,1,9,NaN,1618.269758,0
1,12,-9,0,0,0,-17,0,9,13.0,1560.283091,0
2,14,-4,0,0,0,-2,0,2,NaN,1594.035405,1
3,2,0,1,0,0,8,1,1,NaN,1653.478999,0
4,21,-3,0,0,0,-2,1,10,NaN,1641.639350,1


In [117]:
df = data.drop(["AP_rank"], axis = 1)
df = df.fillna(0)

X = df.drop(['upset'], axis=1)
y = df['upset']


In [118]:
#Apply smote
sm = SMOTE()
X_res, y_res = sm.fit_resample(X, y)

In [121]:
X_res.head()

,rank_change,win_loss,OT,OT_num,point_differential,home_game,CONF,opponent_rank,Elo_pregame
0,-1,1,0,0,6,1,9,0.0,1618.269758
1,-9,0,0,0,-17,0,9,13.0,1560.283091
2,-4,0,0,0,-2,0,2,0.0,1594.035405
3,0,1,0,0,8,1,1,0.0,1653.478999
4,-3,0,0,0,-2,1,10,0.0,1641.639350


In [122]:
#Bring it back together 
X = X_res.drop(["rank_change"], axis = 1)
y = X_res["rank_change"]


In [123]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
np.mean(np.array(abs(y_pred - y_test)))

np.float64(2.068527918781726)